In [42]:
!python -m spacy init config config.cfg --pipeline ner --lang en


✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import ast
import spacy
from spacy.tokens import DocBin

In [44]:
data=pd.read_csv('ner.csv')
data.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [45]:
data['POS']=data['POS'].apply(lambda x: ast.literal_eval(x))
data['Tag']=data['Tag'].apply(lambda x: ast.literal_eval(x))

In [46]:
data['Tag'][0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O',
 'O']

In [47]:
data=data.sample(frac=1).reset_index(drop=True)
data.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 16591,Damascus was forced to withdraw its troops ear...,"[NNP, VBD, VBN, TO, VB, PRP$, NNS, RBR, DT, NN...","[B-per, O, O, O, O, O, O, O, O, O, O, O, O, O,..."
1,Sentence: 27355,Ophelia 's winds dropped to about 115 kilomete...,"[NNP, POS, NNS, VBD, TO, IN, CD, NNS, DT, NN, ...","[B-per, O, O, O, O, O, O, O, O, O, B-tim, O, O..."
2,Sentence: 33690,AIDS activists in South Africa are expressing ...,"[NNP, NNS, IN, NNP, NNP, VBP, VBG, NN, IN, NN,...","[B-org, O, O, B-geo, I-geo, O, O, O, O, O, O, ..."
3,Sentence: 6957,The coup leaders have promised to hold a gener...,"[DT, NN, NNS, VBP, VBN, TO, VB, DT, JJ, NN, IN...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-t..."
4,Sentence: 45103,"If Goldman Sachs repays the debt , it would no...","[IN, NNP, NNP, VBZ, DT, NN, ,, PRP, MD, RB, VB...","[O, B-per, I-per, O, O, O, O, O, O, O, O, O, O..."


In [48]:
train_data,test_val_data=train_test_split(data,test_size=0.2,random_state=42)
test_data,val_data=train_test_split(test_val_data,test_size=0.5,random_state=42)
print('shape of train data:-',train_data.shape)
print('shape of test data:-',test_data.shape)
print('shape of val data:-',val_data.shape)

shape of train data:- (38367, 4)
shape of test data:- (4796, 4)
shape of val data:- (4796, 4)


In [49]:
def get_spacy_doc(data,type):
  nlp=spacy.blank('en')
  db=DocBin()
  for index,row in data.iterrows():
    text=row['Sentence']
    tags=row['Tag']
    doc=nlp.make_doc(text)
    ents=[]
    i=0
    while i < len(tags):
      label=tags[i]
      if label.startswith("B-"):
        start=doc[i].idx
        end=start + len(doc[i])
        label_c=label[2:]
        j = i + 1
        while j < len(tags) and tags[j].startswith("I-") and tags[j][2:]==label_c:
          end = doc[j].idx + len(doc[j])
          j += 1

        span=doc.char_span(start,end,label=label_c)
        if span is not None:
          ents.append(span)
        i = j
      else:
        i += 1
    doc.ents=ents
    db.add(doc)
  db.to_disk(f'ner_{type}.spacy')

In [50]:
get_spacy_doc(train_data,'train')
get_spacy_doc(test_data,'test')
get_spacy_doc(val_data,'val')

In [51]:
nlp=spacy.blank('en')
doc_bin=DocBin().from_disk('ner_train.spacy')
docs=list(doc_bin.get_docs(nlp.vocab))
doc=docs[0]
print("Text:- ",doc.text)
print('Entities:- ')
for ent in doc.ents:
  print(f'{ent.text} :- {ent.label_}')

Text:-  Already the poorest country in the Western Hemisphere with 80 % of the population living under the poverty line and 54 % in abject poverty , the damage to Port-au-Prince caused the country 's GDP to contract an estimated 5.1 % in 2010 .
Entities:- 
Western Hemisphere :- org
Port :- geo
estimated :- tim


In [52]:
!python -m spacy train config.cfg --output ./output --gpu-id 0 --paths.train ner_train.spacy --paths.dev ner_val.spacy


ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     56.15    0.00    0.00    0.00    0.00
  0     200        309.81   3227.10   51.73   54.21   49.46    0.52
  0     400        192.58   2592.54   62.64   63.60   61.70    0.63
  0     600        217.53   2780.69   66.16   68.61   63.89    0.66
  0     800        297.37   3229.70   67.50   70.32   64.90    0.67
  0    1000        386.83   3639.45   68.12   69.66   66.64    0.68
  0    1200        478.65   4610.28   71.65   74.26   69.21    0.72
  0    1400        499.12   5136.65   71.74   73.11   70.41    0.72
  0    1600        592.75   

In [53]:
!python -m spacy evaluate output/model-best ./ner_test.spacy --output metrics.json

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     100.00
NER P   78.71 
NER R   75.89 
NER F   77.27 
SPEED   28860 


=============================== NER (per type) ===============================

           P       R       F
geo    77.42   82.75   79.99
tim    79.98   75.89   77.88
org    74.91   61.26   67.40
gpe    89.85   87.37   88.59
per    73.68   70.31   71.96
art    60.00    7.14   12.77
eve    56.25   25.00   34.62
nat   100.00   29.17   45.16

✔ Saved results to metrics.json
